# FUSE: Balance of Plant with ThermalSytems_Models.JL


This notebook provides examples for using the GA FUSE Balance of Plant (**BOP**) Actor and the ThermalSystems_Models.jl (**TSM**) package. 

**TSM** is a thermodynamic component-based modeling package for simulating physical systems in which there is energy transfer between multiple energy domains. Currently there are component libraries for ideal gas, isothermal liquid, and multiphase fluid domains. In a given fluid circuit, all 'flow' connected components must be within the same domain. Energy can be coupled between circuits of which through shaft power or heat transfer elements.

The actor which implements the TSM model is the **ActorThermalPlant** which can be controlled by an **ActorBalanceOfPlant** instance or by the user. 


---

### Thermal Plant Actor (act.ActorThermalPlant) Structure

**Parameters**

``model::Symbol`` - The model to construct, acceptable options are :rankine, :brayton

**Actor Data**

Basic
* ``dd::IMASDD.dd{Float64}``
* ``par::FUSE.FUSEparameters__ActorThermalPlant{Float64}``
* ``buildstatus::Bool`` - Boolean to track whether or not plant has been fully constructed (set true after first step)

Component and subsystem Information
* ``odedict::Dict{Symbol, ODESystem}``  - Dictionary where you can index the plant with the component names 
* ``components::Vector{ODESystem}`` - Vector of components represented by ODESystem objects
* ``connections::Vector{Equation}`` - Vector of all equations which describe the ``fullbuild``
* ``odeparams::Vector{Num}``        - Vector of **ALL** of the parameters for the `fullbuild`` system.

Plant Construction Variables
* ``fullbuild::ODESystem``          - The ODESystem object as constructed
* ``plant::ODESystem`` - The simplified ODESystem version of fullbuild. The ``plant`` is a algebreicly simplified version of the  ``fullbuild`` which improves performance. this is the system that actually gets calculated.
* ``prob::ODEProblem``    - The ODEProblem object

Processing
* ``G::MetaGraphs.MetaDiGraph`` - Full graph representation of the plant
* ``gplot::MetaGraphs.MetaDiGraph`` - Simplified graph with additional aesthetic nodes for plotting
* ``sym2var::Dict{Symbol, SymbolicUtils.BasicSymbolic{Real}}``  - Dictionary where a symbol key maps to the actual **variable object** with the same name.
* ``var2val::Dict{SymbolicUtils.BasicSymbolic{Real}, Float64}`` - Dictionary wheere a ***variable object*** key maps to the default value of theat parameter or variable.
* ``optpar::Vector{Symbol}`` - Vector the **Optimizable** parameters for the plant model
* ``x::Vector{Float64}``     - Current values for the parameters in ``optpar``
* ``u::Vector{Float64}``     - Current heat load vector (W) where u = [$\dot{Q}_{breeder}$,$ \dot{Q}_{divertor}$, $\dot{Q}_{first wall}$]

### TSM Conventions

**Unit Convention**
|    Variable          | Symbol  | Unit     |Notes|
|----------------------|:------:|:-----------:|:-----------:|
|Temperature|$T$|K|Converted to °C before added to dd|
|Pressure|$P$|Bar|
|Mass flow rate| $\dot{m}$|kg/s|
|Specific Enthalpy|$h$|J/kg|
|Specific Entropy|$s$|J/kg/K|
|Specific Volume|$v$|m^3/kg|
|Specifc heat at constant pressure|$c_p$|kJ/kg/K|$c_p = (\frac{dh}{dT})_{P=const} = T(\frac{ds}{dT})_{P=const}$|
|Specifc heat at constant volume|$c_v$|kJ/kg/K|$c_v = (\frac{du}{dT})_{v=const} = T(\frac{ds}{dT})_{v=const}$|
|Vapour Fraction (multiphase)|$x$|-|
|Mechanical Work and Power|$W, \dot{W}$|J, W|
|Thermal Work and Power|$Q, \dot{Q}$|J, W|

**Sign Convention** for all conserved quantites (mass, work, heat, energy) <br>
|Sign|Transfer Direction|Examples|
|----------------------|:------:|:-----------:|
|(-)| Transfer out of the system|Turbine power, Heat Rejection, $\dot{m}_{out}$|
|(+)| Transfer into the system|Pumping power, Heat addition, $\dot{m}_{in}$ |


**Docs** [ThermalSystems_Models.jl docs](https://legendary-adventure-k6ye327.pages.github.io/UTILITIES.html) 


### Tutorial Outline

* High level use of the FUSE Balance of Plant Actor
* Using an individual ActorThermalPlant instance
* Internals
* Optimizing plant parameters
* Constructing models using the TSM package

### Dependencies 

In [2]:
# FUSE Initialization
using Revise, FUSE, Plots, Logging, Printf
gr(); FUSE.logging(Logging.Info);
Logging.disable_logging(Logging.Warn)

# TSM initialization
using ThermalSystem_Models, ModelingToolkit, DifferentialEquations
TSM     = ThermalSystem_Models;
TSMD    = TSM.Dynamics;
Steam   = TSMD.Steam;
Gas     = TSMD.Gas;
Liq     = TSMD.Liq;

## Basic Use Case - Running with FUSE
Initialize Fuse FPP Case & running Prerequisite Actors required for the Balance of Plant actor. 
Heat loads are taken directly form the Divertor, Blanket, and Neutronics actors. 
For basic use cases, you can simply run the FUSE.ActorBalanceOfPlant(dd,act) to run the bop simulation off of the current dd data.

#### Running from Balance of Plant Actor

In [ ]:
# Initialization
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars);
dd       = FUSE.init(ini, act; do_plot = false);

# Pre-Requisite Actors
FUSE.ActorCXbuild(dd, act);
FUSE.ActorNeutronics(dd, act; do_plot=false);
act.ActorBlanket.minimum_first_wall_thickness = 0.02;
FUSE.ActorBlanket(dd, act, verbose=false);
FUSE.ActorDivertors(dd,act);

In [ ]:
# Brayton
act.ActorThermalPlant.model = :brayton
FUSE.ActorBalanceOfPlant(dd,act; do_plot = false);
@show dd.balance_of_plant.thermal_efficiency_plant
@show dd.balance_of_plant.thermal_efficiency_cycle
@show dd.balance_of_plant.power_plant.power_electric_generated
display(IMAS.freeze(dd.balance_of_plant.power_plant.system[2]))

In [ ]:
# Rankine (default)
act.ActorThermalPlant.model = :rankine
FUSE.ActorBalanceOfPlant(dd,act; do_plot = true);
@show dd.balance_of_plant.thermal_efficiency_plant
@show dd.balance_of_plant.thermal_efficiency_cycle
@show dd.balance_of_plant.power_plant.power_electric_generated
display(IMAS.freeze(dd.balance_of_plant.power_plant.system[2]))

#### Running the ThermalPlantActor.jl independently

Still modifies dd, only missing information is the power needs from the other actors.

In [ ]:
# Brayton, Actor only
act.ActorThermalPlant.model  = :brayton                   
plantActor = FUSE.ActorThermalPlant(dd, act; doplot = true, verbose = true);

In [ ]:
# Rankine, Actor only
act.ActorThermalPlant.model  = :rankine                      
plantActor = FUSE.ActorThermalPlant(dd, act; doplot = true, verbose = true);   

## TSM Internals


#### Component Based Model Approach
The plant object is represented by a MetaGraph object where the vertices represent components and the Edges represent, thermal, mechanical, or fluid power flow. Each vertex has an internal sys::ODESystem which describes the components functionality. Components have internal port elements which correspond to one of the domains thermal, mechanical, flow. Internal equations describe the relationship between each port. 
These ODESystems can be accessed directly from:

<code> plantActor.odedict[component_name::Symbol] => ODESystem object </code> 

In [ ]:
# Example for the wall_circulator
@show TSMD.showsys(plantActor.odedict[:wall_circulator])
@show plantActor.odedict[:wall_circulator]

Besides the obvious components, the Plant system also includes energy reservoirs to track cummulative performance. All individual components can be referenced in the dict: "Actor.odedict" (for ease in reference) where the symbol name corresponds to the system Energy reservoirs are used to supply and recieve energy from the BOP plant. 

There are 3 default reservoirs, Electric, HotUtility (Heat Supply), and ColdUtility (heat rejection). These can be initialized by calling ``TSMD.default_energy_sys()``, which returns: 

* <code>energy_sys::Vector{ODESystem}</code> a Vector with 3 ODE systems representing HotUtility, ColdUtility, and Electric Reservoirs. 
* <code>sts::Vector{Num}</code>  The states associated with these systems, η_cycle and η_bop are the thermal effeciencies 
* <code>edict::Dictionary{Symbol => ODESystem}</code>A dictionary with system_name => ODESystem object .


To view all of the components in the system:

In [ ]:
TSMD.showsys(plantActor.fullbuild);

<h3> Port names</h3>

Components with SISO flow (1 stream input, 1 stream output) have inlet and outlet flow nodes identified by 
<code>p</code> and <code>n</code><br>
All thermal nodes are identified by <code>q</code><br>
All shaft power nodes are identified by <code>w</code>

In [ ]:
# Example for steam boiler, which has flow inlet and outlet, as well as aheat flow node
TSMD.showsys(plantActor.odedict[:steam_boiler]);

### Results
The ODESolution (anonymous function) is stored and updated within the graph of the actor. To get the solution object, call:

In [ ]:
sol = FUSE.getsol(plantActor);  # solution object

#### Observing results for specific components
The solution produced when the actor is run is stored inside of the Actor.G and the Actor.gplot. To find results for individual components, call: <code>sol(var::Variable) = solution for that variable </code>, var has to be the variable object, not just a symbol. However the <code>Actor.odedict</code> can return the parent system object and variables can be referenced from there.
Example: <code> sol(Actor.odedict[:steam_boiler].n.T) = outlet temperature of the boiler </code>

In [ ]:
@show sol(plantActor.odedict[:steam_boiler].n.T);       # Boiler Outlet Temperature
@show sol(plantActor.odedict[:breeder_circulator].w.Ẇ); # Breeder ciruclater pumping power
@show sol(plantActor.odedict[:wall_circulator].p.ṁ);    # First wall coolant flow rate

#### High Level Variables


The plant system object has high level variables which can be accessed directly rather than . Besides the obvious components, the Plant system also includes Utility reservoirs to track cummulative performance. Key variables will always be:

Energy Balance (components)
>Electric.$\dot{W} $,  Net electrical power for the BOP system <br>
HotUtility.$\dot{Q}$, Total heat flow into the BOP system<br>
ColdUtility.$\dot{Q}$, Total heat flow out of the BOP<br>

Thermal Effeciency (Auxillary variables, not tied to a component) 

$\eta_{therm} = \dot{W}_{net}/\dot{Q}_{in} = 1 - \dot{Q}_{out}/\dot{Q}_{in}$

>$\eta_{bop}$, Thermal effeciency of the entire BOP Plant<br>
>$\eta_{cycle}$ Thermal effeciency of the cycle only <br>

> NOTE: $ \eta_{cycle} > \eta_{bop} $ Always since the power generation occurs within the primary the cycle. 

In [ ]:
util = [plantActor.odedict[:Electric].Ẇ,
        plantActor.odedict[:HotUtility].Q̇,
        plantActor.odedict[:ColdUtility].Q̇];
        
println("Power (MW)")
true && [(@printf "%-18s %+-8.2f\n" string(util[i]) sol(util[i])/1e6) for i =1:3]

println("\nEfficiency")
@printf "η_cycle (%%) %8.5f \n" sol(:η_cycle)
@printf "η_plant (%%) %8.5f\n"  sol(:η_bop)

println("\nVerifying relationships for η_bop")
@printf "Ẇnet/Q̇h = %8.5f \n" abs(sol(util[1])/sol(util[2]))
@printf "1-Ql/Qh = %8.5f \n" 1-abs(sol(util[3])/sol(util[2]))
plantActor.G[:Electric,:name]

#### Editing configuration and rerunning

Get data from plantActor

In [ ]:
opt_actor = deepcopy(plantActor);
usym = [:Qbreeder,:Qdivertor,:Qwall]

# Tunable parameters/states for the plant system
opt_x   = Dict(opt_actor.optpar .=> opt_actor.x)

# External input (heat loading)
opt_u   = opt_actor.u # [Qbreeder, Qdivertor, Qwall]

#### Updated data points

In [ ]:
# change flow rates and temperatures
opt_x[:steam_ṁ] = 240;
opt_x[:breeder_heat₊Tout] = 1000;

# change the heat loading
@show opt_u = opt_u + rand(3) .* 100e6;

xnew = [opt_x[p] for p in opt_actor.optpar]

opt_actor.u = opt_u;
opt_actor.x = xnew;

newsol = FUSE.plant_wrapper(opt_actor)

datapoints = [ opt_actor.odedict[:steam_turbine].hp.w.Ẇ,
                opt_actor.odedict[:steam_turbine].lp.w.Ẇ,
                opt_actor.odedict[:wall_circulator].w.Ẇ,
                opt_actor.odedict[:divertor_circulator].w.Ẇ,
                opt_actor.odedict[:breeder_circulator].w.Ẇ,
                opt_actor.odedict[:inter_loop_circulator].w.Ẇ,
                opt_actor.odedict[:steam_hp_pump].w.Ẇ,
                opt_actor.odedict[:steam_lp_pump].w.Ẇ]

oldsol =  sol.(datapoints)
nsol   =  newsol.(datapoints)

println("Original")
display(oldsol)
@show sum(oldsol)
@show sol(plantActor.odedict[:Electric].Ẇ)
println("After modifications");
display(nsol);
@show sum(nsol);
@show newsol(opt_actor.odedict[:Electric].Ẇ);


### Reuse for optimization

In [ ]:
# restart, copy actor
opt_actor = deepcopy(plantActor);
opt_u = opt_actor.u;

# Variables to be optimized (Tunable parameters/states for the plant system)
# x0 = opt_actor.x (same as below, but copy and pasted for the reader)
#       cycle ṁ, loop ṁ,   loop Tmin,   wTmin,    wTmax,  divTmin, divTmax, brdrTmin,   brdrTmax
x0 =   [250.0,   205.0,    493.15,      523.15,  723.15,  573.15,  923.15,  773.15,     1073.15];
lb   = [10.0,    10.0,     350.00,      350.00,  601.00,  350.00,  601.00,  600.00,     901.00];
ub   = [300.0,   300.0,    600.00,      600.00,  950.00,  600.00,  1000.00, 900.00,     1300.00];
#      [  flow rates  ]  [loop Temp]   [   wall temp    ] [ divertor temp  ][ breeder temp     ]

# Relevant, System output variable required for the objective function, 
# this is a simple case where we will just optimize the total electric power produced
yvars = [opt_actor.odedict[:Electric].Ẇ, opt_actor.plant.η_bop, opt_actor.plant.η_cycle];

# anonymous object function which will act on the sol(yvars)
yfunc(y) = -(y[1])/100e6

# Constraint enforcing
function xcons!(x,lb,ub)
    # x = vector{T}
    # lb is lowerbounds
    # ub is upper bounds
    # cons is vector of tuples 
    # (a,b) where a < b

    @assert length(x) == length(lb) == length(ub) "Uneven vector lengths in xcons"
    for (i,xi) in enumerate(x)
        xi < lb[i] ? x[i] = lb[i] : nothing
        xi > ub[i] ? x[i] = ub[i] : nothing
    end
end

# general optimization funciton which can be adapted into an anonymous handle to optimize any set of vatriables in x0
# x0_idx are the indices in x0 which should be optimized
function gen_optfunc(x,x0_idx)
    xrep = deepcopy(x0);
    xrep[x0_idx] .= x
    xcons!(xrep,lb,ub)
    opt_actor.x = xrep;
    return FUSE.plant_wrapper(opt_actor,yvars,yfunc)
end

function eval_optfunc(x,x0_idx)
    xrep = deepcopy(x0);
    xrep[x0_idx] .= x
    xcons!(xrep,lb,ub)
    opt_actor.x = xrep;
    return FUSE.plant_wrapper(opt_actor,yvars)
end

In [ ]:
# restart, copy actor
opt_actor = deepcopy(plantActor);
opt_u = opt_actor.u;
# Variables to be optimized (Tunable parameters/states for the plant system)
# x0 = opt_actor.x (same as below, but copy and pasted for the reader)
#       cycle ṁ, loop ṁ,   loop Tmin,   wTmin,    wTmax,  divTmin, divTmax, brdrTmin,   brdrTmax
x0 =   [100.0,   205.0,    493.15,      523.15,  723.15,  573.15,  923.15,  773.15,     1073.15];
lb   = [10.0,    10.0,     350.00,      350.00,  601.00,  350.00,  601.00,  600.00,     901.00];
ub   = [300.0,   300.0,    600.00,      600.00,  950.00,  600.00,  1000.00, 900.00,     1300.00];
#      [  flow rates  ]  [loop Temp]   [   wall temp    ] [ divertor temp  ][ breeder temp     ]

# Relevant, System output variable required for the objective function, 
# this is a simple case where we will just optimize the total electric power produced
yvars = [opt_actor.odedict[:Electric].Ẇ, opt_actor.plant.η_bop, opt_actor.plant.η_cycle];

# anonymous object function which will act on the sol(yvars)
yfunc(y) = -(y[1])/100e6

# Constraint enforcing
function xcons!(x,lb,ub)
    # x = vector{T}
    # lb is lowerbounds
    # ub is upper bounds
    # cons is vector of tuples 
    # (a,b) where a < b

    @assert length(x) == length(lb) == length(ub) "Uneven vector lengths in xcons"
    for (i,xi) in enumerate(x)
        xi < lb[i] ? x[i] = lb[i] : nothing
        xi > ub[i] ? x[i] = ub[i] : nothing
    end
end

# general optimization funciton which can be adapted into an anonymous handle to optimize any set of vatriables in x0
# x0_idx are the indices in x0 which should be optimized
function gen_optfunc(x,x0_idx)
    xrep = deepcopy(x0);
    xrep[x0_idx] .= x
    xcons!(xrep,lb,ub)
    opt_actor.x = xrep;
    return FUSE.plant_wrapper(opt_actor,yvars,yfunc)
end

function eval_optfunc(x,x0_idx)
    xrep = deepcopy(x0);
    xrep[x0_idx] .= x
    xcons!(xrep,lb,ub)
    opt_actor.x = xrep;
    return FUSE.plant_wrapper(opt_actor,yvars)
end

### Optimzation Examples


In [ ]:
using Optim

Optimizing 
>$Electric.\dot{W}$ (net power production) 

while only varying the:
> intermediate loop $\dot{m}$ <br>
> cycle $\dot{m}$

In [ ]:
mflow_opt_idx   = [1,2];
x0_opt          = x0[mflow_opt_idx];
mflow_opt_func(x) = gen_optfunc(x,mflow_opt_idx);
xr = Optim.optimize(mflow_opt_func,x0_opt,NelderMead());

In [ ]:
xf = Optim.minimizer(xr);
yf = eval_optfunc(xf,mflow_opt_idx);
for i =1:length(yvars)
    @printf "%-16s = %+-8.4g\n" string(yvars[i]) yf[i]
end

Optimizing (this one can take a while)

>$Electric.\dot{W}$ (net power production) 

while varying all variables:
> Cycle $\dot{m}$ <br>
> Intermediate loop $\dot{m}$ <br>
> Intermediate loop $T_{min}$ <br>
> Wall $T_{min}$ and $T_{max}$ <br>
> Divertor $T_{min}$ and $T_{max}$ <br>
> Breeder $T_{min}$ and $T_{max}$ 


Results in
>Electric₊Ẇ(t)    = +3.55e+08 <br>
>η_bop(t)         = +0.3949  <br>
>η_cycle(t)       = +0.4657 <br>

In [ ]:
full_opt_idx   = 1:9;
x0_opt          = x0[full_opt_idx]
mflow_opt_func(x) = gen_optfunc(x,full_opt_idx);
xr = Optim.optimize(mflow_opt_func,x0_opt,NelderMead());

In [ ]:
xf = Optim.minimizer(xr);
yf = eval_optfunc(xf,full_opt_idx);
for i =1:length(yvars)
    @printf "%-16s = %+-8.4g\n" string(yvars[i]) yf[i]
end
opt_actor.x
# Rankine
# 286.7191787095004
# 300.0
# 350.0
# 350.0
# 950.0
# 350.0
# 1000.0
# 674.6482463241312
# 1136.1241752571002

# Brayton
# 300.0
# 300.0
# 350.0
# 350.0
# 950.0
# 350.0
# 1000.0
# 900.0
# 1300.0

## Using Thermal System Models Package
### Constructing the BOP System
Symbolic time variable

In [ ]:
ModelingToolkit.@variables t ;

#### Initialize energy reservoirs

<p>Energy reservoirs are used to supply and recieve energy from the BOP plant. <br>
There are 3 default reservoirs, Electric, HotUtility (Heat Supply), and ColdUtility (heat rejection) <br>
These can be initialized by calling TSMD.default_energy_sys(), which returns:</p>
<ol>
<li> energy_sys :: Vector{ODESystem} <ul><li>Vector with 3 ODE systems representing HotUtility, ColdUtility, and Electric Reservoirs </li></ul></li>
<li> sts :: Vector{Num} <ul><li> The states associated with these systems, η_cycle and η_bop are the thermal effeciencies </li></ul></li>
<li> edict :: Dictionary{Symbol => ODESystem} <ul><li>     A dictionary with system_name => ODESystem object </li> </li></ul></li>
</ol>


In [ ]:
energy_sys, sts, edict = TSMD.default_energy_sys();
η_cycle, η_bop = sts;   # add them to current namespace 

### TSM Initializations

#### Initialize the 3 cooling circuits for the first wall, blanket, and breeder

The cooling circuits are initialized in the following format:

Inputs: Max and Min fluid temperature and heat load


> sys, connections, params, dict = TSMD.XXX_circuit(; load, Tmin, Tmax)
>
> - sys, Vector{ODESystem} ----------> Components
> - connections::Vector{Equation} ---> The equations which connect the components (conservation laws)
> - params::Vector{Num} -------------> Parameter names              
> - dict:Dictionary --------------------> where name::Symbol -> ODESystem
>

In [ ]:
Tmax_wall   = 450 + 273.15; # Maximum cooling temperature for first wall
Tmin_wall   = 250 + 273.15; # Minimum cooling temperature for first wall
Tmax_div    = 650 + 273.15; # Maximum cooling temperature for Divertor
Tmin_div    = 300 + 273.15; # Minimum cooling temperature for Divertor
Tmax_breeder = 800 + 273.15; # Maximum cooling temperature for Breeder blanket
Tmin_breeder = 500 + 273.15; # Minimum cooling temperature for Breeder blanket


# Wall circuit, Helium
wall_sys,     wall_connections,     wparams, wdict  = TSMD.wall_circuit(; load = wall_heat_load, Tmin = Tmin_wall, Tmax = Tmax_wall);

# Divertor circuit, Helium
divertor_sys, divertor_connections, dparams, ddict  = TSMD.divertor_circuit(; load = divertor_heat_load, Tmin = Tmin_div, Tmax = Tmax_div);

# Breeder Circuit (PbLi
breeder_sys,  breeder_connections,  bparams, bdict  = TSMD.breeder_circuit(; load = breeder_heat_load, Tmin = Tmin_breeder, Tmax = Tmax_breeder);

# Initialize The intermediate loop with parameters:
Nhx = 4;                        # Nhx = the number of heat exchangers to add to the loop, 4: 3 to connect to cooling loops, 1 to connect to primary power cycle
flowrate  = 200;                # mass flow rate of the intermediate loop (kg/s)
Tmin_interloop =  200 + 273.15; # minimum temperature for inter loop (Kelvin)
inter_loop_sys, inter_loop_connections, iparams, idict = TSMD.intermediate_loop(; Nhx = Nhx, flowrate = flowrate, Tmin = Tmin_interloop);

# Initialize Primary Cycle
cycle_flowrate = 250;      # kg/s
ηpump          = 0.7;      # isentropic effeciency of the pump
ηturbine        = 0.95;    # Isentropic effeciency of the turbine
steam_systems, steam_connections, sparams, sdict = TSMD.feedwater_rankine(; flowrate = cycle_flowrate, ηpump = ηpump, ηturbine = ηturbine);

### Connecting the BOP Plant

#### Create Heat Exchangers

In [ ]:
# Create heat exchangers which will couple the indepentent loops
@named hx1 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx1],
    A = wdict[:wall_hx],
    returnmode = :eq,
);

@named hx2 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx2],
    A = ddict[:divertor_hx],
    returnmode = :eq,
);

@named hx3 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx3],
    A = bdict[:breeder_hx],
    returnmode = :eq,
);

@named boilhx = TSMD.S2G_HeatExchanger(
    A = sdict[:steam_boiler],
    B = idict[:inter_loop_hx4],
    returnmode = :eq,
);


In [ ]:
# Connect all energy reservoirs to external interfacing components
energy_connections = vcat(
    TSMD.work_connect(
        edict[:Electric],
        wdict[:wall_circulator].w,
        ddict[:divertor_circulator].w,
        bdict[:breeder_circulator].w,
        idict[:inter_loop_circulator].w,
        sdict[:steam_hp_pump].w,
        sdict[:steam_lp_pump].w,
        sdict[:steam_turbine].hp.w,
        sdict[:steam_turbine].lp.w,
    ),
    TSMD.heat_connect(
        edict[:HotUtility],
        wdict[:wall_heat].q,
        ddict[:divertor_heat].q,
        bdict[:breeder_heat].q,
    ),
    TSMD.heat_connect(
        edict[:ColdUtility],
        wdict[:wall_relief].q,
        ddict[:divertor_relief].q,
        bdict[:breeder_relief].q,
        idict[:inter_loop_relief].q,
        sdict[:steam_condensor].q,
    ),
    η_cycle ~ 1 - abs(sdict[:steam_condensor].q.Q̇ / sdict[:steam_boiler].q.Q̇),
    η_bop ~ abs(edict[:Electric].Ẇ / edict[:HotUtility].Q̇),
);

# Create vector of all parameters
plant_params = vcat(wparams, dparams, bparams, iparams, sparams);

# Create total vector for all connecting equations for flow connected components and the energy_connections
plant_connections = vcat(
    steam_connections,
    inter_loop_connections,
    wall_connections,
    divertor_connections,
    breeder_connections,
    energy_connections,
);

# Create total vector for all components
plant_systems = vcat(steam_systems, inter_loop_sys, wall_sys, divertor_sys, breeder_sys, energy_sys);

# add heat exchanger equations to plant_connections
push!(plant_connections, hx1...);
push!(plant_connections, hx2...);
push!(plant_connections, hx3...);
push!(plant_connections, boilhx...);

### Creating and solving the system

In [ ]:
# Create total ODESystem for the plant
@named sys = ODESystem(plant_connections, t, sts, plant_params; systems = plant_systems);

# Check DOF and problem size
TSMD.system_details(sys);

# Simplify using ModelingToolkit's model reduction methods
simple_sys = structural_simplify(sys);


In [ ]:
# Create ODEProblem
tspan    = (0.0, 100)
ode_prob = ODEProblem(simple_sys, [], tspan);
ode_sol  = DifferentialEquations.solve(ode_prob, Rodas4());
# DifferentialEquations.solve()
# Anonymous function for ODE solution functions
soln(v) = ode_sol[v][end]

# Create graph
utility_vector = [:HotUtility, :ColdUtility, :Electric];
GG = TSMD.system2metagraph(sys, utility_vector; soln = soln, verbose = false);

### Plotting


In [ ]:
gcopy = TSMD.create_plot_graph(GG;toignore = [:steam_condensor]);
xLayReqs, vSortReqs, xs, ys, paths, lay2node = TSMD.layers_to_force!(gcopy);
TSMD.initialize_plot_props!(gcopy, lay2node,xs,ys,paths);
TSMD.add_plot_elments!(gcopy);
TSMD.set_default_node_prop!(gcopy, :height, 1.0);
xLayReqs, vSortReqs, xs, ys, paths, lay2node = TSMD.layers_to_force!(gcopy);
x, y = TSMD.setVerticalSpacing!(gcopy; vspan = 40.0);
TSMD.setLayerWidth!(gcopy; pad = 2.5, verbose = false);
xLayReqs, vSortReqs, xs, ys, paths, lay2node = TSMD.layers_to_force!(gcopy);
TSMD.edgeroute_nodes(gcopy; voff = 0.1);
TSMD.set_plot_props!(gcopy);
syslabs = TSMD.get_prop(gcopy, :system_labels)
sysnamedict = Dict([
    "cycle_" => "Brayton Helium",
    "steam_" => "Feedwater Rankine",
    "divertor_" => "Divertor Helium",
    "breeder_" => "Breeder PbLi",
    "inter_loop_" => "Inter. loop Helium",
    "wall_" => "first wall helium",
    "ColdUtility" => "Sink",
    "HotUtility" => "Fusion Core",
])


p = TSMD.plotplant(
    gcopy;
    numbering = false,
    mode = :path,
    nsize = 2.0,
    compnamesubs = (
        "enfw" => "en\nfw",
        "_" => "\n",
        "circulator" => "pump",
        "hotutility" => "Fusion\nCore",
        "coldutility" => "Sink",
        "relief" => "Trim\ncooler",
        "condensor" => "Cool\nHX",
    ),
    compnameattr = (:right,  4),
    compnamerot = 90,
    sysnamedict = sysnamedict,
    legpad = 0.5,
    legwid = 13,
    legheight = 1.5,
    legoffset = 2.0,
    pathattr = (
        linewidth = 1,
        marker = false,
        markersize = 0.0,
        markercolor = :red,
        alpha = 0.7,
        legend = false,
    ),
    figattr = (
        grid = false,
        aspect_ratio = :equal,
        showaxis = false,
        xlim = [-15, 75],
        ylim = [-21, 21],
        xticks = [0, 1, 2, 3, 4, 5, 6, 7],
        plot_title = "Feedwater Rankine",
        plot_titlefonthalign = :hcenter,
        plot_titlefontvalign = :bottom,
        dpi = 200,
        plot_titlevspan = .0001,
    ), #aspect_ratio = :equal
);

In [ ]:
para_ = parameters(simple_sys)
para  = TSMD.variable2symbol(parameters(simple_sys))
parav = ode_prob.p
parad = Dict{Any,Any}()

parad = Dict(para[i] => parav[i]  for i =1:length(para)); # Symbol to value
paraa = Dict(para_[i] => parav[i] for i =1:length(para)); # Variable to value (for prob)
parar = Dict(para[i] => para_[i]  for i=1:length(para));  # symbol to variable

getval(x) = paraa[parar[x]]
getvar(x) = parar[x]
sys.breeder_supply

@show typeof(parad)
@show typeof(paraa) 

### Modifying existing system


### Convert to graph for plotting and ease of access

#### Plot

## Individual Components

### Compressor

**For gas cooling circuits**

$$
 ({T_1},{P_1}) \to Compression \to  (T_2, P_2)\\ 
$$

$$
(\frac{T_2}{T_1}) = (\frac{P_2}{P_1})^{(k-1)/k} = (\frac{v_1}{v_2})^k \\
h_2-h_1 = c_p \cdot (T_2-T_1) \\
Ẇ = ṁ \Delta h =ṁ \cdot c_p \cdot (T_2 - T_1) \\
ẇ =Ẇ / ṁ= c_p \cdot (T_2 - T_1)
$$



### Computed Analytically:

In [ ]:
## Helium props
cp  = 5.1926e3
cv  = 3.1156e3
ṁ   = 10
k   = cp/cv

# inlet conditions
T1  = 30 + 273.15;                       # 30 C in Kelvin
P1  = 1; # pressure in bar, use TSMD.convert_pressure(xx,:pa) to convert
rp  = 3.0;                               # compression ratio
η   = 0.9;                               # compressor isentropic efficiency

P2      = rp*P1;
T2s     = T1 * rp^((k-1)/k)
ws      = cp * (T2s-T1);
wa      = ws/η
dTact   = wa/cp;
T2a     = T1+dTact;
fspec   =  "%-15s = %-4.2f" 

fprint(n,v) = TSMD.@printf  "%-25s = %-4.2f \n" n v
fprintl(n,v) = TSMD.@printf  "%-30s = %-4.2f \n" n v
println("Inputs")
fprint("Inlet Temperture (C)", T1-273.15)
fprint("Inlet Pressure (bar)", P1)
fprint( "Mass Flow Rate (kg/s)",ṁ)
fprint( "Compressor Efficiency (%)",η*100)
fprint( "Compression Ratio", rp)
println("")
println("Intermediate Calculations")
fprintl("Isen. T2 (C)", T2s-273.15)
fprintl("Isen. specific work (kJ/kg)", ws/1e3)

println("")
println("Results")
fprintl("Actual specific work (J/kg)", wa/1e3)
fprintl("Actual compressor power (kW)", wa*ṁ/1e3)
fprintl("Actual T2 (C)",T2a-273.15)
fprintl("Outlet Pressure (bar)",P2)

##### Compressor model ThermalSystem_models.j'
##### Formulation 1: Explanatory

In [ ]:
## Using the PassiveThermoCompressor model
ModelingToolkit.@variables t

# Initialize Gas Component
@named compressor = Gas.PassiveThermoCompressor(η = η);

## define BC
bc = [compressor.p.ṁ ~ ṁ;       # inlet mass flow
        compressor.p.Φ ~ 0;     # Stream energy - excursion variable
        compressor.p.P ~ P1;    # inlet pressure
        compressor.n.P ~ P2;    # outlet pressure
        compressor.p.T ~ T1];   # inlet temperature

@named sys = ODESystem(bc, t, [], []; systems = [compressor]);
simp_sys = structural_simplify(sys);
tspan = (0.0, 1.0);
prob = ODEProblem(simp_sys, [], tspan);
TSMD.showsol([compressor],prob);            # returns a dict with symbol keys that map to the actual ODE system objects

### Using TSM Compressor models for automatic boundary conditions

In [ ]:
ModelingToolkit.@variables t
@named supply         = Gas.SinglePortReservoir(P = P1, T = T1);
@named flowsource     = Gas.GasFlowSource(Ṁ = ṁ)
@named compressor     = Gas.PassiveThermoCompressor(η = η);
@named fixed_pressure = Gas.SetPressure(P = P2)

connections =vcat(Gas.gas_connect(supply.n,flowsource.p),
                Gas.gas_connect(flowsource.n,compressor.p),
                Gas.gas_connect(compressor.n,fixed_pressure.p))

systems = [supply,flowsource,compressor,fixed_pressure];

@named sys = ODESystem(connections, t, [], []; systems = systems);
TSMD.system_details(sys)
simp_sys = structural_simplify(sys);
tspan = (0.0, 1.0);
prob = ODEProblem(simp_sys, [], tspan);
TSMD.showsol(systems,prob);

In [ ]:
sol = DifferentialEquations.solve(prob);
soln(v) = sol[v][end];

### Full workflow

In [ ]:
## Create system
ModelingToolkit.@variables t

energy_sys, sts, edict = TSMD.default_energy_sys();
wall_sys, wall_connections, wparams, wdict =
    TSMD.wall_circuit(; load = 100e6, Tmin = 250 + 273.15, Tmax = 450 + 273.15);
divertor_sys, divertor_connections, dparams, ddict =
    TSMD.divertor_circuit(; load = 150e6, Tmin = 300 + 273.15, Tmax = 650 + 273.15);
breeder_sys, breeder_connections, bparams, bdict =
    TSMD.breeder_circuit(; load = 250e6, Tmin = 500 + 273.15, Tmax = 800 + 273.15)
inter_loop_sys, inter_loop_connections, iparams, idict =
    TSMD.intermediate_loop(; Nhx = 4, flowrate = 200, Tmin = 220 + 273.15);
steam_systems, steam_connections, sparams, sdict = TSMD.feedwater_rankine(; flowrate = 550, ηpump = 0.7, ηturbine = 0.95);
η_cycle, η_bop = sts;

energy_con = vcat(
    TSMD.work_connect(
        edict[:Electric],
        wdict[:wall_circulator].w,
        ddict[:divertor_circulator].w,
        bdict[:breeder_circulator].w,
        idict[:inter_loop_circulator].w,
        sdict[:steam_hp_pump].w,
        sdict[:steam_lp_pump].w,
        sdict[:steam_turbine].hp.w,
        sdict[:steam_turbine].lp.w,
    ),
    TSMD.heat_connect(
        edict[:HotUtility],
        wdict[:wall_heat].q,
        ddict[:divertor_heat].q,
        bdict[:breeder_heat].q,
    ),
    TSMD.heat_connect(
        edict[:ColdUtility],
        wdict[:wall_relief].q,
        ddict[:divertor_relief].q,
        bdict[:breeder_relief].q,
        idict[:inter_loop_relief].q,
        sdict[:steam_condensor].q,
    ),
    η_cycle ~ 1 - abs(sdict[:steam_condensor].q.Q̇ / sdict[:steam_boiler].q.Q̇),
    η_bop ~ 1 - abs(edict[:ColdUtility].Q̇ / edict[:HotUtility].Q̇),
);

params = vcat(wparams, dparams, bparams, iparams, sparams);
connections = vcat(
    steam_connections,
    inter_loop_connections,
    wall_connections,
    divertor_connections,
    breeder_connections,
    energy_con,
);
plant_systems =
    vcat(steam_systems, inter_loop_sys, wall_sys, divertor_sys, breeder_sys, energy_sys)

@named hx1 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx1],
    A = wdict[:wall_hx],
    returnmode = :eq,
);

@named hx2 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx2],
    A = ddict[:divertor_hx],
    returnmode = :eq,
);

@named hx3 = TSMD.Gen_HeatExchanger(
    B = idict[:inter_loop_hx3],
    A = bdict[:breeder_hx],
    returnmode = :eq,
);

@named boilhx = TSMD.Gen_HeatExchanger(
    A = sdict[:steam_boiler],
    B = idict[:inter_loop_hx4],
    returnmode = :eq,
);

push!(connections, hx1...);
push!(connections, hx2...);
push!(connections, hx3...);
push!(connections, boilhx...);

@named sys = ODESystem(connections, t, sts, params; systems = plant_systems);
utility_vector = [:HotUtility, :ColdUtility, :Electric];
TSMD.system_details(sys);
simple_sys = structural_simplify(sys);
tspan = (0.0, 1.0)
prob = ODEProblem(simple_sys, [], tspan)
sol = DifferentialEquations.solve(prob);
soln(v) = sol[v][end];
GG = TSMD.system2metagraph(sys, utility_vector; soln = soln, verbose = false);